In [16]:
from dotenv import load_dotenv
import os

load_dotenv() 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [17]:
from todos_service import tools ,tasks
tools

[StructuredTool(name='get_tasks', description='Retrieves a list of tasks, optionally filtered by provided criteria.\n\nArgs:\n    filters (Optional[Dict]): A dictionary of filter criteria where keys are task attributes.\n\nReturns:\n    List[Dict]: A list of tasks matching the filters, or all tasks if no filters are provided.', args_schema=<class 'langchain_core.utils.pydantic.get_tasks'>, func=<function get_tasks at 0x000001BE1B0E3C40>),
 StructuredTool(name='add_task', description='Adds a new task to the task list.\n\nArgs:\n    title (str): The title of the task.\n    description (str): A detailed description of the task.\n    task_type (str): The type/category of the task.\n    start_date (str): The start date of the task.\n    end_date (str): The end date of the task.\n    status (str): The current status of the task (e.g., pending, completed).\n\nReturns:\n    Dict: The newly created task object with a unique ID.', args_schema=<class 'langchain_core.utils.pydantic.add_task'>, fun

In [18]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
model=ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0)
agent = create_tool_calling_agent(model, tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
tasks

[]

In [20]:
from flask import Flask, request, jsonify
from flask_cors import CORS


app = Flask(__name__)
CORS(app)

In [21]:
def current_response(response):
    return response["messages"][len(response["messages"])-1].content

In [22]:
@app.route('/agent', methods=['GET'])
def get_welcom():
    return "--------------welcom to agent ✨------------------"

@app.route('/agent', methods=['POST'])
def run_agent():
    """הפעלה של ה-Agent עם LangGraph"""

    user_prompt = request.json.get('prompt')
    response = agent_executor.invoke({"input": user_prompt})
    return {'response': response['output']} 

In [23]:
if __name__ == '__main__':
   app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.150.192:5000
Press CTRL+C to quit




> Entering new AgentExecutor chain...

Invoking: `add_task` with `{'title': 'Buy new shoes for son', 'description': 'Purchase a new pair of shoes for my son.', 'task_type': 'Shopping', 'start_date': '2023-10-10', 'end_date': '2023-10-15', 'status': 'pending'}`


{'id': '10dc43c1-f95d-4c58-b962-36da4c97e204', 'title': 'Buy new shoes for son', 'description': 'Purchase a new pair of shoes for my son.', 'task_type': 'Shopping', 'start_date': '2023-10-10', 'end_date': '2023-10-15', 'status': 'pending'}

127.0.0.1 - - [11/Mar/2025 16:05:58] "POST /agent HTTP/1.1" 200 -


I have added the task to buy new shoes for your son to the task list. Here are the details:

- **Title:** Buy new shoes for son
- **Description:** Purchase a new pair of shoes for my son.
- **Task Type:** Shopping
- **Start Date:** October 10, 2023
- **End Date:** October 15, 2023
- **Status:** Pending

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_tasks` with `{}`


[{'id': '10dc43c1-f95d-4c58-b962-36da4c97e204', 'title': 'Buy new shoes for son', 'description': 'Purchase a new pair of shoes for my son.', 'task_type': 'Shopping', 'start_date': '2023-10-10', 'end_date': '2023-10-15', 'status': 'pending'}]

127.0.0.1 - - [11/Mar/2025 16:06:03] "POST /agent HTTP/1.1" 200 -


You have the following task:

- **Title:** Buy new shoes for son
- **Description:** Purchase a new pair of shoes for my son.
- **Task Type:** Shopping
- **Start Date:** October 10, 2023
- **End Date:** October 15, 2023
- **Status:** Pending

> Finished chain.
